## Import

In [2]:
import random
import pandas as pd
import numpy as np
import os
import re
import glob
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from tqdm.auto import tqdm
import timm

import warnings
warnings.filterwarnings(action='ignore') 

/opt/conda/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

## Hyperparameter Setting

In [4]:
CFG = {
    'IMG_SIZE':224,
    'EPOCHS':10,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':32,
    'SEED':41
}

## Fixed RandomSeed

In [5]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

## Data Pre-processing

In [6]:
# all_img_list = glob.glob('./data/train/*/*')

In [7]:
# df = pd.DataFrame(columns=['img_path', 'label'])
# df['img_path'] = all_img_list
# df['label'] = df['img_path'].apply(lambda x : str(x).split('/')[3]) # baseline code 3으로 수정해야함!!

In [16]:
df = pd.read_csv('./data/train.csv')
df['filename'] = df['filename'].apply(lambda x : os.path.join('./Data/Training_whole/PNG',x))

In [17]:
df['filename'] 

0            ./Data/Training_whole/PNG/울음_12.png
1            ./Data/Training_whole/PNG/울음_11.png
2            ./Data/Training_whole/PNG/울음_16.png
3            ./Data/Training_whole/PNG/울음_17.png
4            ./Data/Training_whole/PNG/울음_21.png
                          ...                   
3452    ./Data/Training_whole/PNG/걸레받이수정_160.png
3453    ./Data/Training_whole/PNG/걸레받이수정_214.png
3454    ./Data/Training_whole/PNG/걸레받이수정_284.png
3455    ./Data/Training_whole/PNG/걸레받이수정_198.png
3456    ./Data/Training_whole/PNG/걸레받이수정_157.png
Name: filename, Length: 3457, dtype: object

In [18]:
train, val, _, _ = train_test_split(df, df['label'], test_size=0.3, stratify=df['label'], random_state=CFG['SEED'])

In [19]:
train['label'].value_counts()

훼손         983
오염         416
걸레받이수정     215
꼬임         147
터짐         113
곰팡이        102
오타공         99
몰딩수정        91
면불량         69
석고수정        40
들뜸          38
피스          36
창틀,문틀수정     19
울음          15
이음부불량       12
녹오염         10
가구수정         8
틈새과다         4
반점           2
Name: label, dtype: int64

In [20]:
val['label'].value_counts().values

훼손         422
오염         179
걸레받이수정      92
꼬임          63
터짐          49
오타공         43
곰팡이         43
몰딩수정        39
면불량         30
석고수정        17
들뜸          16
피스          15
창틀,문틀수정      8
울음           7
이음부불량        5
가구수정         4
녹오염          4
반점           1
틈새과다         1
Name: label, dtype: int64

## Label-Encoding

In [21]:
le = preprocessing.LabelEncoder()
train['label'] = le.fit_transform(train['label'])
val['label'] = le.transform(val['label'])

In [22]:
val

,id,filename,label
3271,3271,./Data/Training_whole/PNG/걸레받이수정_14.png,1
1487,1487,./Data/Training_whole/PNG/훼손_440.png,18
3150,3150,./Data/Training_whole/PNG/걸레받이수정_260.png,1
2380,2380,./Data/Training_whole/PNG/오염_147.png,10
1689,1689,./Data/Training_whole/PNG/훼손_1077.png,18
...,...,...,...
181,181,./Data/Training_whole/PNG/터짐_124.png,15
2180,2180,./Data/Training_whole/PNG/오염_108.png,10
654,654,./Data/Training_whole/PNG/훼손_1051.png,18
2792,2792,./Data/Training_whole/PNG/꼬임_195.png,3


## CustomDataset

In [23]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        
        image = cv2.imread(img_path)
        
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        
        if self.label_list is not None:
            label = self.label_list[index]
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)

In [24]:
train_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

test_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

In [25]:
train_dataset = CustomDataset(train['filename'].values, train['label'].values, train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

val_dataset = CustomDataset(val['filename'].values, val['label'].values, test_transform)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [26]:
len(train_dataset)

2419

In [27]:
len(val_dataset)

1038

In [28]:
next(iter(train_loader))

[tensor([[[[ 0.3994,  0.3481,  0.4337,  ...,  0.4851,  0.5536,  0.5022],
           [ 0.4166,  0.4508,  0.4679,  ...,  0.5536,  0.5707,  0.5536],
           [ 0.4508,  0.4508,  0.4337,  ...,  0.5364,  0.5193,  0.5193],
           ...,
           [ 0.3652,  0.3994,  0.3652,  ...,  0.3994,  0.3481,  0.3309],
           [ 0.3481,  0.3823,  0.3823,  ...,  0.3994,  0.3823,  0.3652],
           [ 0.3823,  0.3823,  0.3823,  ...,  0.3652,  0.3481,  0.2967]],
 
          [[ 0.5728,  0.5203,  0.6078,  ...,  0.6078,  0.6779,  0.6254],
           [ 0.5903,  0.6254,  0.6429,  ...,  0.6604,  0.6954,  0.6779],
           [ 0.6254,  0.6254,  0.6078,  ...,  0.6429,  0.6429,  0.6429],
           ...,
           [ 0.6429,  0.6779,  0.6604,  ...,  0.6779,  0.6604,  0.6604],
           [ 0.6254,  0.6604,  0.6604,  ...,  0.6779,  0.6779,  0.6779],
           [ 0.6604,  0.6604,  0.6604,  ...,  0.6604,  0.6604,  0.6078]],
 
          [[ 0.7925,  0.7402,  0.8448,  ...,  0.7576,  0.8274,  0.7751],
           [ 

## Model Define

In [29]:
class BaseModel(nn.Module):
    def __init__(self, num_classes=len(le.classes_)):
        super(BaseModel, self).__init__()
        self.backbone = timm.create_model('efficientnet_b0', pretrained=True, num_classes=num_classes)
        # self.classifier = nn.Linear(1000, num_classes)
        
    def forward(self, x):
        x = self.backbone(x)
        # x = self.classifier(x)
        return x

## Train

In [30]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    
    best_score = 0
    best_model = None
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for imgs, labels in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            output = model(imgs)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val Weighted F1 Score : [{_val_score:.5f}]')
       
        if scheduler is not None:
            scheduler.step(_val_score)
            
        if best_score < _val_score:
            best_score = _val_score
            best_model = model
    
    return best_model

In [31]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, true_labels = [], []

    with torch.no_grad():
        for imgs, labels in tqdm(iter(val_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            pred = model(imgs)
            
            loss = criterion(pred, labels)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
            true_labels += labels.detach().cpu().numpy().tolist()
            
            val_loss.append(loss.item())
        
        _val_loss = np.mean(val_loss)
        _val_score = f1_score(true_labels, preds, average='weighted')
    
    return _val_loss, _val_score

## Run!!

In [32]:
model = BaseModel()
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, threshold_mode='abs', min_lr=1e-8, verbose=True)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)
torch.save(infer_model.module.state_dict(), './models/efficient_net_b0_base')


Epoch [1], Train Loss : [1.67743] Val Loss : [0.95323] Val Weighted F1 Score : [0.68901]



Epoch [2], Train Loss : [0.16739] Val Loss : [0.88447] Val Weighted F1 Score : [0.71648]



Epoch [3], Train Loss : [0.01531] Val Loss : [0.92262] Val Weighted F1 Score : [0.71425]



Epoch [4], Train Loss : [0.00632] Val Loss : [0.92777] Val Weighted F1 Score : [0.71331]



Epoch [5], Train Loss : [0.00394] Val Loss : [0.94095] Val Weighted F1 Score : [0.71233]
Epoch     5: reducing learning rate of group 0 to 1.5000e-04.


...


Epoch [7], Train Loss : [0.00264] Val Loss : [0.95346] Val Weighted F1 Score : [0.70934]



Epoch [8], Train Loss : [0.00232] Val Loss : [0.95939] Val Weighted F1 Score : [0.70739]
Epoch     8: reducing learning rate of group 0 to 7.5000e-05.



Epoch [9], Train Loss : [0.00208] Val Loss : [0.96247] Val Weighted F1 Score : [0.70739]



Epoch [10], Train Loss : [0.00196] Val Loss : [0.96551] Val Weighted F1 Score : [0.70739]


ModuleAttributeError: 'BaseModel' object has no attribute 'module'

In [38]:
torch.save(infer_model.state_dict(), './models/efficient_net_b0_base.pt')

<bound method Module.modules of BaseModel(
  (backbone): EfficientNet(
    (conv_stem): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNormAct2d(
      32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): SiLU(inplace=True)
    )
    (blocks): Sequential(
      (0): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn1): BatchNormAct2d(
            32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): SiLU(inplace=True)
          )
          (se): SqueezeExcite(
            (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (act1): SiLU(inplace=True)
            (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (gate): Sigmoid()
          )
          (conv_pw)

## Inference

In [138]:
test = pd.read_csv('./data/test.csv')

In [146]:
test['img_path'] = test['img_path'].apply(lambda x : os.path.join('./data', '/'.join(x.split('/')[1:])))

In [147]:
test_dataset = CustomDataset(test['img_path'].values, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [148]:
def inference(model, test_loader, device):
    model.eval()
    preds = []
    with torch.no_grad():
        for imgs in tqdm(iter(test_loader)):
            imgs = imgs.float().to(device)
            
            pred = model(imgs)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
    
    preds = le.inverse_transform(preds)
    return preds

In [149]:
preds = inference(infer_model, test_loader, device)

## Submission

In [150]:
submit = pd.read_csv('./data/sample_submission.csv')

In [151]:
submit['label'] = preds

In [152]:
submit.to_csv('./baseline_submit.csv', index=False)